In [1]:
import json
from pathlib import Path

In [2]:
exported_restrictions = Path('ZionandGWMPTest.json')

with exported_restrictions.open("r", encoding="utf-8") as f:
    new_restrictions = json.load(f)
    
print(type(new_restrictions))

<class 'dict'>


In [3]:
len(new_restrictions['features'])

33

In [4]:
from datetime import datetime

# Get the current local date and time
# Get the current time in UTC
now_utc = datetime.utcnow()

# Format using string format codes
current_datetime = now_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

In [5]:
new_restrictions['features'][0]['id'] = str(new_restrictions['features'][0]['attributes']['OBJECTID'])

## Data Sources

- Create a list of dictionaries for each data source

In [18]:
park_list = []
contact_list = []
contact_email_list = []

data_sources = []

for x in range(len(new_restrictions['features'])):
    new_restrictions['features'][x]['id'] = str(new_restrictions['features'][x]['attributes']['OBJECTID'])
    
    park_alpha = new_restrictions['features'][x]['attributes']['park_alpha']
    contact = new_restrictions['features'][x]['attributes']['contact_name']
    
    if contact is None:
        continue

    if new_restrictions['features'][x]['attributes']['park_alpha'] == "ZION":
        contact_email = "Lisa.White@nps.gov"
    else:
        contact_email = "steven_kattula@nps.gov"

    this_data_source = {
        "data_source_id": park_alpha,
        "organization_name": park_alpha,
        "contact_name": contact,
        "contact_email": contact_email, 
        "update_date": str(current_datetime)   
        }
        
    if this_data_source not in data_sources:
        data_sources.append(this_data_source)

    





In [19]:
data_sources

[{'data_source_id': 'GWMP',
  'organization_name': 'GWMP',
  'contact_name': 'Steven Kattula',
  'contact_email': 'steven_kattula@nps.gov',
  'update_date': '2025-11-05T15:26:39Z'},
 {'data_source_id': 'ZION',
  'organization_name': 'ZION',
  'contact_name': 'Lisa White; Lisa_M_White@nps.gov',
  'contact_email': 'Lisa.White@nps.gov',
  'update_date': '2025-11-05T15:26:39Z'}]

## Feed Info

- Create the initial block of the road restriction json that has the overall feed_info



In [29]:
feed_info_key = {   #this stuff look like it stays static except for the date which I can figure out how to update later
    'update_date' : str(current_datetime),
    'publisher' : "National Park Service",
    'contact_name' :"National Park Service" ,
    'contact_email' : "asknps@nps.gov",
    'version' : "4.0",
    'license' : "https://creativecommons.org/publicdomain/zero/1.0/",
    'data_sources' : data_sources}

feed_info_dict = {'feed_info': feed_info_key}

In [30]:
feed_info_key|{"type": "FeatureCollection"}

{'update_date': '2025-11-05T15:26:39Z',
 'publisher': 'National Park Service',
 'contact_name': 'National Park Service',
 'contact_email': 'asknps@nps.gov',
 'version': '4.0',
 'license': 'https://creativecommons.org/publicdomain/zero/1.0/',
 'data_sources': [{'data_source_id': 'GWMP',
   'organization_name': 'GWMP',
   'contact_name': 'Steven Kattula',
   'contact_email': 'steven_kattula@nps.gov',
   'update_date': '2025-11-05T15:26:39Z'},
  {'data_source_id': 'ZION',
   'organization_name': 'ZION',
   'contact_name': 'Lisa White; Lisa_M_White@nps.gov',
   'contact_email': 'Lisa.White@nps.gov',
   'update_date': '2025-11-05T15:26:39Z'}],
 'type': 'FeatureCollection'}

## Create Restrictions

In [42]:
road_names=[]
road_names.append(new_restrictions['features'][0]['attributes']['road_names'])
road_names

['SOUTHDOWN ROAD']

In [ ]:
features = []
restriction_type_list = ["local_access_only", "no_trucks","reduced_width","reduced_height",
                         "reduced_length","reduced_weight", "axle_load_limit","gross_weight_limit",
                         "towing_prohibited","permitted_oversize_loads_prohibited","no_passing"]

for x in range(len(new_restrictions['features'])):

    this_restriction = []
    this_lane = []
    road_names=[]

    id = new_restrictions['features'][x]['attributes']['OBJECTID']
    park_alpha = new_restrictions['features'][x]['attributes']['park_alpha']
    road_names.append(new_restrictions['features'][x]['attributes']['road_names'])
    direction = new_restrictions['features'][x]['attributes']['Direction']
    geometry = new_restrictions['features'][x]['geometry']

    if direction is None:
        direction = "northbound"

    this_core_details = {
        "event_type": "restriction",
        "data_source_id": park_alpha,
        "road_names": road_names,
        "direction": direction
    }

    for y in restriction_type_list:
        restriction_value = new_restrictions['features'][x]['attributes'][y]
        if restriction_value not in [None, "", 0]:
            if y=="reduced_width":
                this_restriction_partial = {
                    "type": "reduced-width",
                    "value": new_restrictions['features'][x]['attributes']['reduced_width_value'],
                    "unit": new_restrictions['features'][x]['attributes']['reduced_width_unit']
                }

            elif y=="reduced_height":
                this_restriction_partial = {
                    "type": "reduced-height",
                    "value": new_restrictions['features'][x]['attributes']['reduced_height_value'],
                    "unit": new_restrictions['features'][x]['attributes']['reduced_height_unit']
                }

            elif y=="reduced_length":
                this_restriction_partial = {
                    "type": "reduced-length",
                    "value": new_restrictions['features'][x]['attributes']['reduced_length_value'],
                    "unit": new_restrictions['features'][x]['attributes']['reduced_length_unit']
                }

            elif y=="reduced_weight":
                this_restriction_partial = {
                    "type": "reduced-weight",
                    "value": new_restrictions['features'][x]['attributes']['reduced_weight_value'],
                    "unit": new_restrictions['features'][x]['attributes']['reduced_weight_unit']
                }
            else:
                this_restriction_partial = {
                    "type": y.replace("_","-")
                } 



            this_restriction.append(this_restriction_partial) 

    this_lane.append({
        "order": 1,
        "status": "open", 
        "type": "general",
        "restrictions": this_restriction
    })

    this_feature = {
        "id": park_alpha,
        "type": "Feature",
        "properties": {"core_details": this_core_details, 
                       "restrictions": this_restriction, 
                       "lanes": this_lane},
        "geometry": geometry, 
        }
        
    if this_feature not in features:
        features.append(this_feature)


In [50]:
updated_feed = {}
updated_feed = feed_info_dict | {"type": "FeatureCollection"} | {"features": features}

In [51]:
path = Path('NPS-Road-Restrictions-Updated.json')

with path.open("w", encoding="utf-8") as f:
    json.dump(updated_feed, f, indent=2, ensure_ascii=False)